In [11]:
!pip install transformers 
!pip install farasapy 
!pip install pyarabic 

In [2]:
!git clone https://github.com/aub-mind/arabert.git

fatal: destination path 'arabert' already exists and is not an empty directory.


In [3]:
#import GPT2LMHeadModel for Text generation and GPT2Tokenizer for tokenizing the text.
from transformers import GPT2TokenizerFast, pipeline
#for base and medium
from transformers import GPT2LMHeadModel
#for large and mega
from arabert.preprocess import ArabertPreprocessor

AraGPT-2 base

In [4]:
MODEL_NAME='aubmindlab/aragpt2-base'
arabert_prep = ArabertPreprocessor(model_name=MODEL_NAME)

model = GPT2LMHeadModel.from_pretrained(MODEL_NAME)
tokenizer = GPT2TokenizerFast.from_pretrained(MODEL_NAME)
generation_pipeline = pipeline("text-generation",model=model,tokenizer=tokenizer)

Downloading:   0%|          | 0.00/843 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/553M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.50M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.52M [00:00<?, ?B/s]

Text prompt

In [5]:
sentence = "شعرت بالسعادة"

In [6]:
input_ids = tokenizer.encode(sentence, return_tensors = 'pt')

In [7]:
input_ids

tensor([[14223,   273, 35098]])

In [8]:
tokenizer.decode(input_ids[0])

'شعرت بالسعادة'

In [9]:
def generate_text(text):
  text_clean = arabert_prep.preprocess(text)
  result = generation_pipeline(text_clean,
                                pad_token_id=tokenizer.eos_token_id,
                                num_beams=10,
                                max_length=100,
                                top_p=0.9,
                                repetition_penalty = 3.0,
                                no_repeat_ngram_size = 3)[0]['generated_text']
  return result

In [14]:
generate_text(sentence)

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1890: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


'شعرت بالسعادة التي كانت تغمرني عندما علمت أن هناك من هو أكثر سعادة مني في الدنيا ، ومن هو أكثر حزنا مني في الآخرة .لكن ما الذي حدث لي بعد ذلك ؟ لم يكن الأمر سهلا على الإطلاق ، فقد كان كل شيء مبهرا بالنسبة لي ، ولم أكن أتخيل أني سأحصل على هذا القدر من السعادة إلا إذا كنت قد حصلت على هذه الدرجة من السعادة .ومنذ ذلك الحين وأنا أحظى بسعادة لا مثيل لها ، حتى أنني أصبحت أعيش حياتي بشكل مختلف تماما عما كنت عليه'

GPT2-Small-Arabic

In [ ]:
!pip install -q transformers

In [58]:
from transformers import AutoTokenizer, AutoModelWithLMHead
tokenizer = AutoTokenizer.from_pretrained("akhooli/gpt2-small-arabic")
model = AutoModelWithLMHead.from_pretrained("akhooli/gpt2-small-arabic")

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:925: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [63]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [60]:
sentence2 = "القدس مدينة تاريخية تقع في "

In [61]:
def generate_text(text):
  try:
    input_ids = tokenizer.encode(sentence2, return_tensors='pt').to(device) 
    model.to(device)
    max_length = 100
    #set top_k = 40 and num_return_sequences = 3
    sample_outputs = model.generate(input_ids, pad_token_id=50256,
                                      do_sample=True, 
                                      max_length=max_length, 
                                      min_length=max_length,
                                      top_k=40,
                                      num_return_sequences=5)

    for i, sample_output in enumerate(sample_outputs):
        print(">> Generated text {}\n\n{}".format(i+1, tokenizer.decode(sample_output.tolist())))
        print('\n---')
  except exception as e:
    print("Something Wrong",e)

In [62]:
generate_text(sentence2)

>> Generated text 1

القدس مدينة تاريخية تقع في اهرام في منطقة الجليل الأعلى. وقد ورد ذلك في إنجيل متى، وقال يسوع أن مريم قد تزوجت من الرسول محمد فقال: ما الذي هو الذي يخون له؟ قال: ما الذي هو الذي يخون لنا؟ وقال: ما الذي هو لا يعرفونه؟ قال: فإن الله يأمر على ما ينبغي له أن يأمر عليه، ثم طلب منه أن يعطي في الوقت المناسب، فقال، إن كنت غير قادر على فعل شيء من هذا الكلام. وقد ورد في إنجيل متى،

---
>> Generated text 2

القدس مدينة تاريخية تقع في ادة شمال ولاية ولاية الخرطوم في السودان حيث كانت تسكن في تلك الفترة منطقة الشلالات. تقع الولاية على الحدود السودانية مع دولة السودان حيث تحتوي على عدد من الفنادق التي تعتبر الأكبر في العالم. تضم الولاية ثلاثة متاحف منها : قاعة السياحة القديمة والتي تضم قاعة سينما و مكتبة عامة بالإضافة إلى مجموعة من المتاحف القديمة، وكذلك مجموعة من التحف الأثرية ومنها المتحف البريطاني ومتحف السودان الوطني. تعد مدينة المهدية مركز ناحية دنقلا بحري هام، كما تقع على الحدود الشرقية من السودان التي تضم عددا من المسارح

---
>> Generated text 3

القدس مدينة تاريخية تقع في 

GPT2-Small-Arabic-Poetry

In [64]:
tokenizer= AutoTokenizer.from_pretrained("akhooli/gpt2-small-arabic-poetry")
model = AutoModelWithLMHead.from_pretrained("akhooli/gpt2-small-arabic-poetry")

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:925: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [69]:
sentence3 = "الحب"

In [70]:
def generate_text(text):
  try:
    input_ids = tokenizer.encode(sentence3, return_tensors='pt').to(device) 
    model.to(device)
    max_length = 100
    #set top_k = 40 and num_return_sequences = 3
    sample_outputs = model.generate(input_ids, pad_token_id=50256,
                                      do_sample=True, 
                                      max_length=max_length, 
                                      min_length=max_length,
                                      top_k=40,
                                      num_return_sequences=5)

    for i, sample_output in enumerate(sample_outputs):
        print(">> Generated text {}\n\n{}".format(i+1, tokenizer.decode(sample_output.tolist())))
        print('\n---')
  except exception as e:
    print("Something Wrong",e)

In [71]:
generate_text(sentence3)

>> Generated text 1

الحبوع ولا غرو - ء ولا عار ولا شيق. يذم البخل ولا يروي - ت له في النهود ولا رق. إمام هدى الله في كل حي - لك يا خير العالمين على وتر. هو من يرجو السلامة أو يرى - ك يا خير الورى على كفر. نازعه في كل بيت في الندى - ك لا يرجى له من غير عار. مولاي خذها مني يا خير الورى 

---
>> Generated text 2

الحبالة أم عبدك من نائل - كذاك العنا والسهل أم داثر. رأيت الناس طرا غير أن مراحهم - خيلت في غمد من العباب الثائر. أرى الناس أحشاء أن يسددوا بنعمة - وإن كان ما إن تراه فهو كافر. ولاقى ما أهلكت غير مدافع - من البخل والجود من كل جانب. إذ نحن أناس عن خير نائل - فإن لم

---
>> Generated text 3

الحبنا بها ما قد حنت من الحشا - ولا من يد بيضاء لم تفز يوما. ومجلسنا بين البطان روضها - كما راق من البان الأبيحا. فهي في الحسن ما حنت نياقها - كما راق من البان الأبيحا. وكم قضيتها بعد اللطف بها - ويا ربرب لا تنهى بها المنى. فكم من نعمة بين الأنام بها - وكم من نعمة بين الأنام

---
>> Generated text 4

الحبينا لنا على رغم أنفاسه - كأن عيونهن صوارم. من كل صفراء كحيله - من خمر نواره وعساج. أحبابنا

Fine-tuning the based on @HHShkMohd tweets



In [12]:
!pip install --upgrade simpletransformers tokenizers==0.9.4
!pip install pytorch cpuonly -c pytorch

  Using cached simpletransformers-0.63.6-py3-none-any.whl (249 kB)
  Using cached transformers-4.19.2-py3-none-any.whl (4.2 MB)
  Using cached transformers-4.19.1-py3-none-any.whl (4.2 MB)
  Using cached transformers-4.19.0-py3-none-any.whl (4.2 MB)
  Using cached transformers-4.18.0-py3-none-any.whl (4.0 MB)
  Using cached transformers-4.17.0-py3-none-any.whl (3.8 MB)
  Using cached transformers-4.16.2-py3-none-any.whl (3.5 MB)
  Using cached transformers-4.16.1-py3-none-any.whl (3.5 MB)
  Using cached transformers-4.16.0-py3-none-any.whl (3.5 MB)
  Using cached transformers-4.15.0-py3-none-any.whl (3.4 MB)
  Using cached transformers-4.14.1-py3-none-any.whl (3.4 MB)
  Using cached transformers-4.13.0-py3-none-any.whl (3.3 MB)
  Using cached transformers-4.12.5-py3-none-any.whl (3.1 MB)
  Using cached transformers-4.12.4-py3-none-any.whl (3.1 MB)
  Using cached transformers-4.12.3-py3-none-any.whl (3.1 MB)
  Using cached transformers-4.12.2-py3-none-any.whl (3.1 MB)
  Using cached tra

In [15]:
! head -n 5 HHShkMohd_tweets.csv

﻿"أثناء جولتي اليوم في سوق السفر العربي ٢٠٢٢ … والذي تشارك فيه ١٥٠٠ جهة من ١٥٨ دولة .. المعارض تعود .. السياحة تتعافى.. العقار يصعد .. الاستثمار يتسارع .. التصدير يصل لمستويات جديدة .. 
الحياة تعود لدبي بحمد الله.. ودبي تقود عودة الحياة للعالم .. https://t.co/rMce14PEt3"
مجلس دبي هدفه قيادة المشاريع التحولية بالإمارة .. وقراراتنا اليوم كبيرة .. ومهمة .. ومستقبلية وتحمل الخير لمواطنينا .. واعتمادي بعد الله على حمدان ومكتوم في متابعة كافة هذه التحولات الجديدة التي سترسم مستقبلاً أفضل وأجمل لأجيالنا القادمة بإذن الله .. https://t.co/aOAd16lWs7
وتم اليوم توقيع اتفاقيات أداء مدراء مسار البنية التحتية مع مفوض البنية التحتية .. وهم بلدية دبي ودائرة الأراضي وبرنامج الإسكان … لديهم مهام كبيرة .. وتغييرات إيجابية سريعة .. ونعول عليهم لإنجاز طموحاتنا خلال الفترة المقبلة .. https://t.co/hXKjlbts9E
كما كلفنا مفوض البنية التحتية الأخ مطر الطاير بوضع خطة متكاملة لتطوير أرياف وبراري دبي مثل الهباب واللسيلي والفقع والمرموم وغيرها.. وتحويلها لمناطق تنموية ووجهات سياحية وثقافية بارزة.. والارتقاء بمستوى خ

In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,accuracy_score, classification_report

data = pd.read_csv(r"HHShkMohd_tweets.csv")
print(data.head())
print(data.sample(5))

  أثناء جولتي اليوم في سوق السفر العربي ٢٠٢٢ … والذي تشارك فيه ١٥٠٠ جهة من ١٥٨ دولة .. المعارض تعود .. السياحة تتعافى.. العقار يصعد .. الاستثمار يتسارع .. التصدير يصل لمستويات جديدة .. \nالحياة تعود لدبي بحمد الله.. ودبي تقود عودة الحياة للعالم .. https://t.co/rMce14PEt3
0  مجلس دبي هدفه قيادة المشاريع التحولية بالإمارة...                                                                                                                                                                                                                           
1  وتم اليوم توقيع اتفاقيات أداء مدراء مسار البني...                                                                                                                                                                                                                           
2  كما كلفنا مفوض البنية التحتية الأخ مطر الطاير ...                                                                                                                                    

In [25]:
punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ''' + string.punctuation

nltk.download('stopwords')
stop_words = stopwords.words()

arabic_diacritics = re.compile("""
                             ّ    | # Shadda
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)

def preprocess(text):
    
    #remove punctuations
    translator = str.maketrans('', '', punctuations)
    text = text.translate(translator)
    
    #remove Tashkeel
    text = re.sub(arabic_diacritics, '', text)
    
    #remove longation
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)

    #remove URL
    text = re.sub(r"http\S+", "", text)

    #remove mentions and hashtags
    mentions = re.findall("@([a-zA-Z0-9_]{1,50})", text)
    hashtags = re.findall("#([a-zA-Z0-9_]{1,50})", text)
    text = re.sub("@[A-Za-z0-9_]+","", text)
    text = re.sub("#[A-Za-z0-9_]+","", text)

    text = ' '.join(word for word in text.split() if word not in stop_words)

    return text
    
    data['Feed'] = data['Feed'].apply(preprocess)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [28]:
print(data.head(5))

  أثناء جولتي اليوم في سوق السفر العربي ٢٠٢٢ … والذي تشارك فيه ١٥٠٠ جهة من ١٥٨ دولة .. المعارض تعود .. السياحة تتعافى.. العقار يصعد .. الاستثمار يتسارع .. التصدير يصل لمستويات جديدة .. \nالحياة تعود لدبي بحمد الله.. ودبي تقود عودة الحياة للعالم .. https://t.co/rMce14PEt3
0  مجلس دبي هدفه قيادة المشاريع التحولية بالإمارة...                                                                                                                                                                                                                           
1  وتم اليوم توقيع اتفاقيات أداء مدراء مسار البني...                                                                                                                                                                                                                           
2  كما كلفنا مفوض البنية التحتية الأخ مطر الطاير ...                                                                                                                                    

In [29]:
import csv
import random

train = []
test = []
eval = []

users = {
    'HH', '[HHShkMohd] '
}

for users in [['HH', '[HHShkMohd] ']]:
  f = open(r"HHShkMohd_tweets.csv", "r")
   
  rdr = csv.reader(f, delimiter='\t')
  lines = 0
  for line in rdr:
      lines += 1
      if line == 1:
        continue
      if len(line) == 1:
        if random.random() > 0.4:
          train.append(arabert_prep.preprocess(line[0])  + '<|endoftext|>')
        elif random.random() > 0.2:
          test.append(arabert_prep.preprocess(line[0])  + '<|endoftext|>')
        else:
          eval.append(arabert_prep.preprocess(line[0])  + '<|endoftext|>')

In [30]:
train[-1]

'العشرية القادمة هي عشرية الانطلاق بكل ثقة للمستقبل بكل تغيراته وتحدياته وتنافسيته . # القمة _ العالمية _ للحكومات<|endoftext|>'

In [31]:
open('./train.txt', 'w').write("\n".join(train))
open('./test.txt', 'w').write("\n".join(test))
open('./eval.txt', 'w').write("\n".join(test))
len(train)

1420

In [32]:
from simpletransformers.language_modeling import LanguageModelingModel, LanguageModelingArgs

train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "train_batch_size": 8,
    "num_train_epochs": 10,
    "fp16": False,
    "mlm": False,
}

ft_model = LanguageModelingModel('gpt2', 'aubmindlab/aragpt2-base', args=train_args)

ImportError: ignored

In [33]:
ft_model.tokenizer.add_tokens(["[HHShkMohd]"])
ft_model.model.resize_token_embeddings(len(ft_model.tokenizer))

NameError: ignored

In [34]:
ft_model.train_model("./train.txt", eval_file="./test.txt")

NameError: ignored

In [ ]:
ft_model.eval_model("./eval.txt")

In [ ]:
ft_model.tokenizer.save_pretrained("./users")

In [ ]:
ft_model.model.save_pretrained("./users")

In [ ]:
from simpletransformers.language_generation import LanguageGenerationModel
model = LanguageGenerationModel("gpt2", "./users")

In [ ]:
prompt = "مدينة دبي"

In [ ]:
model.tokenizer.encode('[HHShkMohd] ' + prompt)

In [ ]:
model.generate('[HHShkMohd] ' + prompt)

In [ ]:
model.generate('[HHShkMohd] ' + prompt, { 'max_length': 50 })